In [1]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets , transforms
import matplotlib.pyplot as plt
import torch.nn as nn

In [10]:
dataset_folder = './ImgFolderDataset'


# Define transformations (resizing, converting to tensor, normalization)
transform = transforms.Compose([
    transforms.Resize((80, 80)),  # Resize images to 32x32 (or use any other size)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

In [11]:
# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root=dataset_folder, transform=transform)


In [12]:
train_data = DataLoader(dataset,batch_size=10,shuffle=True)

In [13]:
class customNN(nn.Module):
    def __init__(self,input_size, hidden_size, num_classes):
        super(customNN, self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.fc3 = nn.Linear(hidden_size,hidden_size)
        self.fc4 = nn.Linear(hidden_size,hidden_size)
        self.fc_out = nn.Linear(hidden_size,num_classes)

        self.relu = nn.ReLU()
    
    def forward(self,x):
        x = x.view(x.size(0),-1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc_out(x)
        return x

In [14]:
input_size = 80 * 80 * 3
hidden_size = 128
num_class = 4
model = customNN(input_size=input_size,hidden_size=hidden_size,num_classes=num_class)


In [15]:
import torch.optim as optim

# 5. Define the Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [16]:
num_epoch = 10

for epoch in range(num_epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_data:
        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        _ , predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss/len(train_data)
    accuracy = 100 * correct/total
    print(f'Epoch [{epoch+1}/{num_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')



Epoch [1/10], Loss: 1.2897, Accuracy: 23.81%
Epoch [2/10], Loss: 1.5722, Accuracy: 42.86%
Epoch [3/10], Loss: 1.0955, Accuracy: 47.62%
Epoch [4/10], Loss: 0.9794, Accuracy: 76.19%
Epoch [5/10], Loss: 0.6699, Accuracy: 90.48%
Epoch [6/10], Loss: 0.4905, Accuracy: 85.71%
Epoch [7/10], Loss: 0.2466, Accuracy: 90.48%
Epoch [8/10], Loss: 0.1319, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0979, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0301, Accuracy: 100.00%


In [17]:
torch.save(model.state_dict(),'./nn/customdatanew')
print("Model saved as './nn/customdatanew'")

Model saved as './nn/customdatanew'


In [18]:
class_names = dataset.classes
class_names

['brad_pit', 'decaprio', 'johny_deep', 'keauna_reaves']

In [19]:
torch.save(model,'./nn/fullmodel')


In [20]:
mymodel = torch.load('./nn/fullmodel')

C:\Users\dhira\AppData\Local\Temp\ipykernel_11472\811624659.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mymodel = torch.load('./nn/fullmodel')


In [28]:
from PIL import Image
def find_img(imgpath,mymodel):
    transform = transforms.Compose([
    transforms.Resize((80, 80)),  # Resize images to 32x32 (or use any other size)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])
    img = Image.open(imgpath)
    imp_tensor = transform(img).unsqueeze(0)

    mymodel.eval()
    with torch.no_grad():
        output = model(imp_tensor)
    _ , predicted = torch.max(output,1)
    print(predicted.item())

In [29]:
find_img('./nn/test/a.jpg',mymodel)

3
